In [ ]:
# ==============================================================================
# LangGraph HITL Example 1: Basic Approval Workflow
#
# Description:
# This script demonstrates a simple Human-in-the-Loop (HITL) workflow using
# LangGraph. The graph generates a short, creative sentence and then pauses
# to ask a human for approval. The graph's execution path changes based on
# whether the human input is "approve" or "reject".
#
# This example is designed to be run in a Jupyter Notebook or a similar
# interactive environment where `input()` can be used.
#
# Required Libraries:
# pip install langchain langchain_core langchain_openai langgraph pytest
#
# Environment Setup:
# You need to have an OpenAI API key set as an environment variable named
# `OPENAI_API_KEY`.
# ==============================================================================

import os
import pytest
from typing import TypedDict, Annotated, Literal
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import StateGraph, END
import uuid
import subprocess
import sys

# --- Set up OpenAI API Key ---
# Make sure to set your OpenAI API key as an environment variable.
# For example: os.environ["OPENAI_API_KEY"] = "your_key_here"
if 'OPENAI_API_KEY' not in os.environ:
    print('WARNING: OPENAI_API_KEY not found in environment variables. The script may fail.')
    # You can uncomment the next line and add your key for quick testing
    # os.environ["OPENAI_API_KEY"] = "sk-..."


# ==============================================================================
# 1. Problem Statement
# ==============================================================================
#
# The task is to build a simple content creation workflow where an AI generates
# a piece of text (e.g., a creative sentence for a marketing campaign), and a
# human supervisor must approve or reject it before it's finalized. This ensures
# that only human-approved content proceeds to the "end" state.
#


# ==============================================================================
# 2. Graph State Definition
# ==============================================================================
#
# Define the state that will be passed between the nodes of the graph.
# It contains the generated text and the decision from the human.
#
class BasicGraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        generation: The text generated by the language model.
        human_decision: The decision made by the human ('approve' or 'reject').
    """

    generation: str
    human_decision: Literal['approve', 'reject']


# ==============================================================================
# 3. Graph Nodes and Conditional Logic
# ==============================================================================
#
# Define the functions that will act as nodes in our graph.
#

# --- Initialize the LLM ---
# We'll use OpenAI's gpt-4o-mini as it is fast and capable for this task.
llm = ChatOpenAI(model='gpt-4o-mini')


def generate_text_node(state: BasicGraphState) -> BasicGraphState:
    """
    Generates a creative sentence using the LLM.

    Args:
        state: The current graph state.

    Returns:
        An updated state with the generated text.
    """
    print('--- Node: Generating Text ---')
    try:
        response = llm.invoke('Generate a single, short, creative sentence for a new coffee brand.')
        generation = response.content
        print(f"   AI Generated: '{generation}'")
        return {'generation': generation}
    except Exception as e:
        print(f'Error during text generation: {e}')
        return {'generation': 'Error: Could not generate text.'}


def human_intervention_node(state: BasicGraphState) -> BasicGraphState:
    """
    Pauses the graph and asks for human input.

    Args:
        state: The current graph state.

    Returns:
        An updated state with the human's decision.
    """
    print('\n--- Node: Human Intervention ---')
    print('Please review the generated text.')
    print(f"   Generated Text: '{state['generation']}'")

    decision = ''
    # Loop until valid input is received
    while decision.lower() not in ['approve', 'reject']:
        decision = input('Your decision (approve/reject): ').strip().lower()
        if decision.lower() not in ['approve', 'reject']:
            print("Invalid input. Please enter 'approve' or 'reject'.")

    print(f'   Human Decision: {decision}')
    return {'human_decision': decision}


def conditional_branch(state: BasicGraphState) -> Literal['approved_end', 'rejected_end']:
    """
    Determines the next step based on the human's decision.

    Args:
        state: The current graph state.

    Returns:
        A string indicating which path to take next.
    """
    print('\n--- Conditional Branch ---')
    decision = state['human_decision']
    if decision == 'approve':
        print('   Routing to: Approved')
        return 'approved_end'
    else:
        print('   Routing to: Rejected')
        return 'rejected_end'


# ==============================================================================
# 4. LangGraph Definition and Execution
# ==============================================================================
#
# Now, we assemble the nodes into a graph.
#


def build_graph():
    """Builds and returns the StateGraph."""
    workflow = StateGraph(BasicGraphState)

    # Add nodes to the graph
    workflow.add_node('generator', generate_text_node)
    workflow.add_node('human_intervention', human_intervention_node)

    # Define the graph's flow
    workflow.set_entry_point('generator')
    workflow.add_edge('generator', 'human_intervention')

    # Add the conditional branch
    workflow.add_conditional_edges(
        'human_intervention',
        conditional_branch,
        {
            'approved_end': END,
            'rejected_end': END,
        },
    )

    # Compile the graph into a runnable object
    app = workflow.compile()
    return app


def run_basic_workflow():
    """Runs the full basic approval workflow."""
    print('==========================================')
    print('= Running Basic HITL Approval Workflow   =')
    print('==========================================')
    app = build_graph()
    final_state = app.invoke({})

    print('\n--- Workflow Finished ---')
    print('Final State:')
    print(f'  - Generated Text: {final_state.get("generation")}')
    print(f'  - Final Decision: {final_state.get("human_decision")}')
    print('==========================================\n')
    return final_state


# ==============================================================================
# 5. Unit Tests using Pytest
# ==============================================================================
#
# To test this, we'll create a separate file named `test_basic_workflow.py`
# and run it with `pytest`. Below is the content for that file.
#
# We use mocking to simulate human input without requiring actual interaction
# during the automated test run.
#
# To run tests:
# 1. Save the code below as `test_basic_workflow.py`.
# 2. Run `pytest test_basic_workflow.py` in your terminal.
#
# --- Content for test_basic_workflow.py ---


# test_basic_workflow.py
def create_test_file():
    test_code = """
import pytest
from unittest.mock import patch
from langgraph_hitl_basic import conditional_branch, human_intervention_node

# Test the conditional logic directly
def test_conditional_branch_approve():
    \"\"\"Tests the 'approve' path of the conditional branch.\"\"\"
    state = {"generation": "Test text", "human_decision": "approve"}
    assert conditional_branch(state) == "approved_end"

def test_conditional_branch_reject():
    \"\"\"Tests the 'reject' path of the conditional branch.\"\"\"
    state = {"generation": "Test text", "human_decision": "reject"}
    assert conditional_branch(state) == "rejected_end"

# Test the human intervention node with mocking
@patch('builtins.input', lambda _: 'approve')
def test_human_intervention_node_approve():
    \"\"\"Tests the human intervention node for an 'approve' input.\"\"\"
    state = {"generation": "A fresh take on your morning cup."}
    result = human_intervention_node(state)
    assert result['human_decision'] == 'approve'

@patch('builtins.input', lambda _: 'reject')
def test_human_intervention_node_reject():
    \"\"\"Tests the human intervention node for a 'reject' input.\"\"\"
    state = {"generation": "A bold new world of flavor."}
    result = human_intervention_node(state)
    assert result['human_decision'] == 'reject'

@patch('builtins.input', side_effect=['invalid', 'approve'])
def test_human_intervention_node_invalid_then_approve(mock_input):
    \"\"\"Tests robustness against invalid user input.\"\"\"
    state = {"generation": "Test text"}
    result = human_intervention_node(state)
    assert result['human_decision'] == 'approve'
    assert mock_input.call_count == 2
"""
    with open('test_basic_workflow.py', 'w') as f:
        f.write(test_code)


def run_tests():
    """Creates and runs the pytest file."""
    print('==========================================')
    print('= Running Unit Tests                     =')
    print('==========================================')
    create_test_file()
    # Use subprocess to run pytest and capture output
    try:
        result = subprocess.run(
            [sys.executable, '-m', 'pytest', 'test_basic_workflow.py'], capture_output=True, text=True, check=True
        )
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print('Pytest execution failed:')
        print(e.stdout)
        print(e.stderr)
    finally:
        # Clean up the test file
        if os.path.exists('test_basic_workflow.py'):
            os.remove('test_basic_workflow.py')
    print('==========================================\n')


# ==============================================================================
# 6. Main Execution Block
# ==============================================================================
if __name__ == '__main__':
    # Note: To run this interactively in a notebook, you might call the
    # functions separately.

    # Run the interactive workflow
    run_basic_workflow()

    # Run the automated tests
    run_tests()


In [ ]:
# ==============================================================================
# LangGraph HITL Example 2: Intermediate Editing Workflow
#
# Description:
# This script builds on the basic example by creating a workflow where an AI
# generates a summary of a document. A human is then given the opportunity
# to review and edit this summary. The final (potentially edited) summary is
# then passed to another AI agent to create a social media post.
#
# This demonstrates how human input can modify the state that subsequent
# nodes in the graph will use.
#
# Required Libraries:
# pip install langchain langchain_core langchain_openai langgraph pytest
#
# Environment Setup:
# Requires an OpenAI API key set as the `OPENAI_API_KEY` environment variable.
# ==============================================================================

import os
import pytest
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
from langgraph.graph import StateGraph, END
import textwrap
import subprocess
import sys

# --- Set up OpenAI API Key ---
if 'OPENAI_API_KEY' not in os.environ:
    print('WARNING: OPENAI_API_KEY not found in environment variables. The script may fail.')
    # You can uncomment the next line and add your key for quick testing
    # os.environ["OPENAI_API_KEY"] = "sk-..."


# ==============================================================================
# 1. Problem Statement
# ==============================================================================
#
# The task is to create a semi-automated content pipeline. Given a long piece
# of text, the system should first generate a concise summary. A human expert
# then reviews this summary for accuracy and nuance, editing it if necessary.
# Finally, the system uses this polished summary to generate a ready-to-publish
# social media post, ensuring the final output aligns with human quality standards.
#


# ==============================================================================
# 2. Graph State Definition
# ==============================================================================
#
# The state now needs to hold the original document, the initial summary,
# the edited summary, and the final social media post.
#
class EditingGraphState(TypedDict):
    """
    Represents the state of our editing graph.

    Attributes:
        document: The original text to be processed.
        initial_summary: The summary generated by the first LLM call.
        edited_summary: The summary after human review and potential edits.
        social_post: The final social media post generated from the edited summary.
    """

    document: str
    initial_summary: str
    edited_summary: str
    social_post: str


# ==============================================================================
# 3. Graph Nodes Definition
# ==============================================================================
#
# We define three nodes: one to summarize, one for human editing, and one
# to create the social media post.
#

# --- Initialize the LLM ---
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.7)


def summarize_document_node(state: EditingGraphState) -> EditingGraphState:
    """
    Generates an initial summary of the document in the state.

    Args:
        state: The current graph state.

    Returns:
        An updated state with the initial summary.
    """
    print('--- Node: Summarizing Document ---')
    document = state['document']
    prompt = [
        SystemMessage(
            content='You are an expert summarizer. Create a concise, one-paragraph summary of the following text.'
        ),
        SystemMessage(content=f'TEXT:\n\n{document}'),
    ]
    try:
        response = llm.invoke(prompt)
        summary = response.content
        print('   AI Generated Summary (Initial):')
        print(textwrap.fill(summary, width=80))
        return {'initial_summary': summary}
    except Exception as e:
        print(f'Error during summarization: {e}')
        return {'initial_summary': 'Error: Could not generate summary.'}


def human_edit_node(state: EditingGraphState) -> EditingGraphState:
    """
    Allows a human to edit the generated summary.

    Args:
        state: The current graph state.

    Returns:
        An updated state with the (potentially) edited summary.
    """
    print('\n--- Node: Human Editing ---')
    initial_summary = state['initial_summary']
    print('Please review the summary. You can either accept it by pressing Enter or provide your own edited version.')
    print('\nInitial Summary:')
    print(textwrap.fill(initial_summary, width=80))

    # Provide a multi-line input prompt
    print(
        '\nEnter your edited summary below. Press Ctrl+D (or Ctrl+Z on Windows) when you are done, or just press Enter to accept the original.'
    )
    edited_lines = []
    try:
        while True:
            line = input()
            edited_lines.append(line)
    except EOFError:
        pass  # This is expected when the user finishes input

    edited_summary = '\n'.join(edited_lines).strip()

    if not edited_summary:
        print('   No changes made. Using original summary.')
        final_summary = initial_summary
    else:
        print('\n   Summary has been edited by human.')
        final_summary = edited_summary

    print('\nFinal Summary to be used:')
    print(textwrap.fill(final_summary, width=80))
    return {'edited_summary': final_summary}


def generate_social_post_node(state: EditingGraphState) -> EditingGraphState:
    """
    Generates a social media post based on the final, human-approved summary.

    Args:
        state: The current graph state.

    Returns:
        An updated state with the generated social media post.
    """
    print('\n--- Node: Generating Social Media Post ---')
    summary_to_use = state['edited_summary']
    prompt = [
        SystemMessage(
            content='You are a social media manager. Based on the following summary, write an engaging and concise Twitter post. Include relevant hashtags.'
        ),
        SystemMessage(content=f'SUMMARY:\n\n{summary_to_use}'),
    ]
    try:
        response = llm.invoke(prompt)
        post = response.content
        print('   AI Generated Social Post:')
        print(textwrap.fill(post, width=80))
        return {'social_post': post}
    except Exception as e:
        print(f'Error during post generation: {e}')
        return {'social_post': 'Error: Could not generate social post.'}


# ==============================================================================
# 4. LangGraph Definition and Execution
# ==============================================================================


def build_graph():
    """Builds and returns the StateGraph for the editing workflow."""
    workflow = StateGraph(EditingGraphState)

    # Add nodes
    workflow.add_node('summarizer', summarize_document_node)
    workflow.add_node('human_editor', human_edit_node)
    workflow.add_node('social_poster', generate_social_post_node)

    # Define the flow (linear in this case)
    workflow.set_entry_point('summarizer')
    workflow.add_edge('summarizer', 'human_editor')
    workflow.add_edge('human_editor', 'social_poster')
    workflow.add_edge('social_poster', END)

    app = workflow.compile()
    return app


def run_editing_workflow():
    """Runs the full intermediate editing workflow."""
    print('==========================================')
    print('= Running Intermediate HITL Editing Workflow =')
    print('==========================================')

    document_text = """
    Quantum computing represents a fundamental shift in computation. Unlike classical
    computers that use bits (0s and 1s), quantum computers use qubits, which can
    exist in a superposition of both 0 and 1 simultaneously. This property, along
    with another quantum phenomenon called entanglement, allows quantum computers to
    explore a vast number of possibilities at once. While still in its early stages,
    quantum computing holds the potential to solve complex problems currently
    intractable for even the most powerful supercomputers, with applications in
    drug discovery, materials science, and financial modeling. However, building
    and controlling stable quantum computers is a massive engineering challenge due
    to their sensitivity to environmental noise, a phenomenon known as decoherence.
    """

    app = build_graph()
    initial_state = {'document': document_text.strip()}
    final_state = app.invoke(initial_state)

    print('\n--- Workflow Finished ---')
    print('Final State Contains:')
    print(f'  - Document: {"Yes" if final_state.get("document") else "No"}')
    print(f'  - Initial Summary: {"Yes" if final_state.get("initial_summary") else "No"}')
    print(f'  - Edited Summary: {"Yes" if final_state.get("edited_summary") else "No"}')
    print(f'  - Social Post: {"Yes" if final_state.get("social_post") else "No"}')
    print('==========================================\n')
    return final_state


# ==============================================================================
# 5. Unit Tests using Pytest
# ==============================================================================
#
# As before, save the code below as `test_editing_workflow.py` and run `pytest`.
#
# --- Content for test_editing_workflow.py ---


def create_test_file():
    test_code = """
import pytest
from unittest.mock import patch
from langgraph_hitl_intermediate import human_edit_node, generate_social_post_node

# Test the human editing node
@patch('builtins.input', side_effect=EOFError) # Simulates pressing Enter with no input
def test_human_edit_node_no_edit(mock_input):
    \"\"\"Tests the case where the human does not edit the summary.\"\"\"
    initial_summary = "This is the original summary."
    state = {"initial_summary": initial_summary}
    result = human_edit_node(state)
    assert result['edited_summary'] == initial_summary

@patch('builtins.input', side_effect=["This is the new, better summary.", EOFError])
def test_human_edit_node_with_edit(mock_input):
    \"\"\"Tests the case where the human provides an edited summary.\"\"\"
    initial_summary = "This is the original summary."
    state = {"initial_summary": initial_summary}
    result = human_edit_node(state)
    assert result['edited_summary'] == "This is the new, better summary."

# Test the social post generation node
# We don't mock the LLM here, but we can test that it uses the correct input field
def test_social_post_uses_edited_summary():
    \"\"\"Ensures the social post node uses the 'edited_summary' from the state.\"\"\"
    # This is more of an integration test for the node's logic
    state = {
        "document": "doc",
        "initial_summary": "initial",
        # The key piece of information for this node
        "edited_summary": "The final, human-approved summary about quantum mechanics.",
        "social_post": ""
    }
    # We can't easily assert the LLM's output, but we can confirm the node runs
    # and produces some output in the correct key.
    result = generate_social_post_node(state)
    assert "social_post" in result
    assert isinstance(result["social_post"], str)
    assert len(result["social_post"]) > 0
    # A more advanced test would mock the LLM call and verify the prompt
"""
    with open('test_editing_workflow.py', 'w') as f:
        f.write(test_code)


def run_tests():
    """Creates and runs the pytest file for the editing workflow."""
    print('==========================================')
    print('= Running Unit Tests                     =')
    print('==========================================')
    create_test_file()
    try:
        result = subprocess.run(
            [sys.executable, '-m', 'pytest', 'test_editing_workflow.py'], capture_output=True, text=True, check=True
        )
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print('Pytest execution failed:')
        print(e.stdout)
        print(e.stderr)
    finally:
        if os.path.exists('test_editing_workflow.py'):
            os.remove('test_editing_workflow.py')
    print('==========================================\n')


# ==============================================================================
# 6. Main Execution Block
# ==============================================================================
if __name__ == '__main__':
    run_editing_workflow()
    run_tests()

In [ ]:
# ==============================================================================
# LangGraph HITL Example 3: Advanced Code Debugging Loop
#
# Description:
# This script implements a sophisticated, cyclical "human-in-the-loop" workflow
# for code generation and debugging. The process is as follows:
# 1. An AI agent generates a Python function based on a problem description.
# 2. A second AI agent generates unit tests (`pytest`) for that function.
# 3. The system executes the tests against the generated code.
# 4. If tests pass, the workflow ends successfully.
# 5. If tests fail, the graph pauses for human intervention. The human is shown
#    the failing code and the test error, and is asked to provide a fix.
# 6. The graph loops back, re-running the tests with the human-corrected code.
#
# This demonstrates a powerful collaborative pattern between an AI and a human
# developer.
#
# Required Libraries:
# pip install langchain langchain_core langchain_openai langgraph pytest
#
# Environment Setup:
# Requires an OpenAI API key set as the `OPENAI_API_KEY` environment variable.
# ==============================================================================

import os
import pytest
import subprocess
import sys
import uuid
import textwrap
from typing import TypedDict, Annotated, List

from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, END

# --- Set up OpenAI API Key ---
if 'OPENAI_API_KEY' not in os.environ:
    print('WARNING: OPENAI_API_KEY not found in environment variables. The script may fail.')
    # You can uncomment the next line and add your key for quick testing
    # os.environ["OPENAI_API_KEY"] = "sk-..."

# ==============================================================================
# 1. Problem Statement
# ==============================================================================
#
# The task is to automate the initial phases of software development. Given a
# clear requirement (e.g., "write a Python function to check if a string is a
# palindrome"), the system should generate both the functional code and the
# corresponding unit tests. Crucially, it must then verify its own work. If the
# verification fails, it should present the problem to a human developer for
# debugging, incorporating the fix into its next attempt. This loop continues
# until a verified, working solution is produced.
#


# ==============================================================================
# 2. Graph State Definition
# ==============================================================================
#
class CodeGenState(TypedDict):
    """
    Represents the state of our code generation and debugging graph.

    Attributes:
        problem_statement: The initial requirement for the code.
        code: The Python code for the function, generated or human-corrected.
        tests: The pytest unit tests for the function.
        test_results: The output from the last pytest run.
        error: A flag indicating if the last test run failed.
        retries: A counter for how many times the loop has run.
    """

    problem_statement: str
    code: str
    tests: str
    test_results: str
    error: bool
    retries: int


# ==============================================================================
# 3. Graph Nodes and Conditional Logic
# ==============================================================================

# --- Initialize LLM ---
# Using a more capable model is better for code generation.
llm = ChatOpenAI(model='gpt-4o', temperature=0.2)


def generate_code_node(state: CodeGenState) -> CodeGenState:
    """Generates the function code based on the problem statement."""
    print('--- Node: Generating Function Code ---')
    prompt = [
        SystemMessage(
            content='You are a senior Python developer. Write a single Python function that solves the following problem. Do not include any example usage or explanations, just the function definition.'
        ),
        HumanMessage(content=state['problem_statement']),
    ]
    try:
        response = llm.invoke(prompt)
        code = response.content.strip()
        # Clean up Markdown code blocks if present
        if code.startswith('```python'):
            code = code[9:]
        if code.endswith('```'):
            code = code[:-3]
        print('   AI Generated Code:\n', textwrap.indent(code, '    '))
        return {'code': code, 'retries': 0}
    except Exception as e:
        print(f'Error during code generation: {e}')
        return {'code': f'# Error: {e}', 'error': True}


def generate_tests_node(state: CodeGenState) -> CodeGenState:
    """Generates unit tests for the given code."""
    print('--- Node: Generating Unit Tests ---')
    prompt = [
        SystemMessage(
            content='You are a quality assurance engineer. Write a set of pytest unit tests for the following Python function. Include a variety of edge cases. Do not include any explanations, just the test code. Assume the function is in a file named `solution.py` and can be imported.'
        ),
        HumanMessage(content=f'Function:\n{state["code"]}\n\nProblem: {state["problem_statement"]}'),
    ]
    try:
        response = llm.invoke(prompt)
        tests = response.content.strip()
        if tests.startswith('```python'):
            tests = tests[9:]
        if tests.endswith('```'):
            tests = tests[:-3]
        print('   AI Generated Tests:\n', textwrap.indent(tests, '    '))
        return {'tests': tests}
    except Exception as e:
        print(f'Error during test generation: {e}')
        return {'tests': f'# Error: {e}', 'error': True}


def execute_tests_node(state: CodeGenState) -> CodeGenState:
    """Saves the code and tests to files and runs pytest."""
    print('\n--- Node: Executing Tests ---')

    # Error handling for missing code/tests
    if not state.get('code') or not state.get('tests'):
        print('   ERROR: Code or tests are missing.')
        return {'error': True, 'test_results': 'Code or tests not generated.'}

    # Create temporary files for the code and tests
    solution_filename = 'solution.py'
    test_filename = f'test_{uuid.uuid4().hex}.py'

    with open(solution_filename, 'w') as f:
        f.write(state['code'])
    with open(test_filename, 'w') as f:
        f.write(state['tests'])

    try:
        # Execute pytest using a subprocess
        result = subprocess.run(
            [sys.executable, '-m', 'pytest', test_filename],
            capture_output=True,
            text=True,
            timeout=30,  # Add a timeout for safety
        )

        if result.returncode == 0:
            print('   ✅ Tests Passed!')
            return {'error': False, 'test_results': result.stdout}
        else:
            print('   ❌ Tests Failed!')
            return {'error': True, 'test_results': result.stdout + result.stderr}

    except subprocess.TimeoutExpired:
        print('   ❌ Test execution timed out.')
        return {'error': True, 'test_results': 'Test execution timed out.'}
    except Exception as e:
        print(f'   ❌ An unexpected error occurred during test execution: {e}')
        return {'error': True, 'test_results': f'An unexpected error occurred: {e}'}
    finally:
        # Clean up the created files
        for filename in [solution_filename, test_filename]:
            if os.path.exists(filename):
                os.remove(filename)


def human_debug_node(state: CodeGenState) -> CodeGenState:
    """Allows a human to debug the failing code."""
    print('\n--- Node: Human Debugging ---')

    # Safety check for max retries
    if state['retries'] >= 3:
        print('   Max retries reached. Exiting workflow.')
        return {'code': state['code'], 'test_results': 'Max retries reached.'}

    print('The generated code failed the tests. Please review and provide a fix.')
    print('\n' + '=' * 80)
    print('Problem Statement:', state['problem_statement'])
    print('-' * 80)
    print('Failing Code:\n')
    print(state['code'])
    print('-' * 80)
    print('Test Results:\n')
    print(state['test_results'])
    print('=' * 80 + '\n')

    print('Enter your corrected code below. Press Ctrl+D (or Ctrl+Z on Windows) when you are done.')
    edited_lines = []
    try:
        while True:
            line = input()
            edited_lines.append(line)
    except EOFError:
        pass

    corrected_code = '\n'.join(edited_lines).strip()

    if not corrected_code:
        print('   No changes provided. Aborting workflow.')
        return {'code': state['code'], 'test_results': 'Aborted by user.'}

    print('\n   Received corrected code from human.')
    return {'code': corrected_code, 'retries': state['retries'] + 1}


def decide_next_step(state: CodeGenState) -> Literal['human_debugger', 'end']:
    """Determines if the loop should continue (debug) or end (success)."""
    print('--- Conditional Branch: Checking Test Results ---')
    if state['error']:
        print('   Routing to: Human Debugger')
        return 'human_debugger'
    else:
        print('   Routing to: End')
        return 'end'


# ==============================================================================
# 4. LangGraph Definition and Execution
# ==============================================================================


def build_graph():
    """Builds and returns the StateGraph for the debugging loop."""
    workflow = StateGraph(CodeGenState)

    workflow.add_node('code_generator', generate_code_node)
    workflow.add_node('test_generator', generate_tests_node)
    workflow.add_node('test_executor', execute_tests_node)
    workflow.add_node('human_debugger', human_debug_node)

    workflow.set_entry_point('code_generator')
    workflow.add_edge('code_generator', 'test_generator')
    workflow.add_edge('test_generator', 'test_executor')

    workflow.add_conditional_edges('test_executor', decide_next_step, {'human_debugger': 'human_debugger', 'end': END})

    # The crucial loop back to the test executor
    workflow.add_edge('human_debugger', 'test_executor')

    app = workflow.compile()
    return app


def run_debugging_workflow():
    """Runs the full advanced debugging workflow."""
    print('==========================================')
    print('= Running Advanced HITL Debugging Workflow =')
    print('==========================================')

    problem = 'Write a Python function `is_palindrome(s: str) -> bool` that checks if a string is a palindrome, ignoring case and non-alphanumeric characters.'

    app = build_graph()
    initial_state = {'problem_statement': problem}

    # The AI might get this right on the first try, or it might fail,
    # which would trigger the human loop.

    final_state = app.invoke(initial_state)

    print('\n--- Workflow Finished ---')
    if not final_state['error']:
        print('✅ Successfully generated and verified the code.')
        print('\nFinal Code:\n', final_state['code'])
    else:
        print('❌ Workflow finished with an error or was aborted.')
        print('\nLast State:\n', final_state)
    print('==========================================\n')
    return final_state


def run_tests():
    """Creates and runs the pytest file for the debugging workflow."""
    print('==========================================')
    print('= Running Unit Tests                     =')
    print('==========================================')
    create_test_file()
    try:
        result = subprocess.run(
            [sys.executable, '-m', 'pytest', 'test_debugging_workflow.py'], capture_output=True, text=True, check=True
        )
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print('Pytest execution failed:')
        print(e.stdout)
        print(e.stderr)
    finally:
        if os.path.exists('test_debugging_workflow.py'):
            os.remove('test_debugging_workflow.py')
    print('==========================================\n')


# ==============================================================================
# 6. Main Execution Block
# ==============================================================================
if __name__ == '__main__':
    run_debugging_workflow()
    run_tests()


In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langgraph.graph import StateGraph, START, END


# Node 1: Generate text using the LLM
def generate_text_node(state):
    llm = ChatOpenAI(temperature=0)
    resp = llm.invoke([HumanMessage(content='Generate a friendly greeting.')])
    state['generated_text'] = resp.content.strip()
    return state


# Node 2: Ask human to approve or reject
def approval_node(state):
    print(f'\nGenerated text: “{state["generated_text"]}”')
    while True:
        ans = input('Approve this text? (y/n): ').strip().lower()
        if ans in ('y', 'n'):
            break
        print("Please enter 'y' or 'n'.")
    state['approved'] = ans == 'y'
    return state


# Build the graph
graph = StateGraph()
graph.add_node('generate', generate_text_node)
graph.add_node('approve', approval_node)
graph.add_node(END, lambda s: s)
graph.add_edge(START, 'generate')
graph.add_edge('generate', 'approve')
# loop or finish
graph.add_edge('approve', END, condition=lambda s: s['approved'])
graph.add_edge('approve', 'generate', condition=lambda s: not s['approved'])

# Execute
result = graph.invoke({})
print('\nFinal state:', result)


ValueError: Must provide state_schema or input and output

In [ ]:
# In [1]: Imports
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langgraph.graph import StateGraph, START, END

# Sample document
DOCUMENT = """
Jupyter Notebook is an open-source web application that allows you to create and share documents
containing live code, equations, visualizations, and narrative text.
"""


# Node 1: Summarize document
def summarize_node(state):
    llm = ChatOpenAI(temperature=0)
    prompt = f'Summarize the following text:\n\n{state["document"]}'
    resp = llm.invoke([HumanMessage(content=prompt)])
    state['summary'] = resp.content.strip()
    return state


# Node 2: Let human edit the summary
def human_edit_node(state):
    print('\nGenerated summary:')
    print(state['summary'])
    while True:
        edited = input('\nEdit summary for accuracy:\n').strip()
        if edited:
            break
        print('Summary cannot be empty.')
    state['edited_summary'] = edited
    return state


# Node 3: Generate keywords from edited summary
def keywords_node(state):
    llm = ChatOpenAI(temperature=0)
    prompt = f'Extract three comma-separated keywords from this summary:\n\n{state["edited_summary"]}'
    resp = llm.invoke([HumanMessage(content=prompt)])
    state['keywords'] = [w.strip() for w in resp.content.split(',') if w.strip()]
    return state


# Build and run the graph
graph = StateGraph()
graph.add_node('summarize', summarize_node)
graph.add_node('edit', human_edit_node)
graph.add_node('keywords', keywords_node)
graph.add_node(END, lambda s: s)

graph.add_edge(START, 'summarize')
graph.add_edge('summarize', 'edit')
graph.add_edge('edit', 'keywords')
graph.add_edge('keywords', END)

result = graph.invoke({'document': DOCUMENT})
print('\nFinal state:', result)


In [ ]:
# In [1]: Imports
import subprocess, os
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langgraph.graph import StateGraph, START, END

# Template for our test suite
TEST_TEMPLATE = """
import generated_code
def test_add():
    assert generated_code.add(2, 3) == 5
"""


# Node 1: Generate code
def generate_code_node(state):
    llm = ChatOpenAI(temperature=0)
    prompt = 'Write a Python module named generated_code.py that defines a function add(a, b) returning their sum.'
    resp = llm.invoke([HumanMessage(content=prompt)])
    state['code'] = resp.content
    return state


# Node 2: Write code & tests to disk
def write_code_node(state):
    with open('generated_code.py', 'w') as f:
        f.write(state['code'])
    with open('test_generated_code.py', 'w') as f:
        f.write(TEST_TEMPLATE)
    return state


# Node 3: Run pytest
def run_tests_node(state):
    result = subprocess.run(['pytest', '-q', 'test_generated_code.py'], capture_output=True, text=True)
    state['tests_passed'] = result.returncode == 0
    state['pytest_output'] = result.stdout + result.stderr
    return state


# Node 4: Human debugging if tests fail
def human_debug_node(state):
    if not state['tests_passed']:
        print('\nPytest output:\n', state['pytest_output'])
        corrected = input('Tests failed. Please provide corrected code for generated_code.py:\n')
        state['code'] = corrected
    return state


# Build looping graph
graph = StateGraph()
graph.add_node('gen_code', generate_code_node)
graph.add_node('write_code', write_code_node)
graph.add_node('run_tests', run_tests_node)
graph.add_node('debug', human_debug_node)
graph.add_node(END, lambda s: s)

graph.add_edge(START, 'gen_code')
graph.add_edge('gen_code', 'write_code')
graph.add_edge('write_code', 'run_tests')
graph.add_edge('run_tests', END, condition=lambda s: s['tests_passed'])
graph.add_edge('run_tests', 'debug', condition=lambda s: not s['tests_passed'])
graph.add_edge('debug', 'write_code')

# Execute
final_state = graph.invoke({})
print('\nFinal result – tests passed?', final_state['tests_passed'])
